In [1]:
# from skbio.diversity import alpha_diversity
# adiv_obs_otus = alpha_diversity('observed_otus', data, ids)
# adiv_obs_otus


In [14]:
import sys
import os
sys.path.insert(0,'/global/homes/b/bpb/repos/blink/lbnl_tools/ToM_Diversity/')
# sys.path.insert(0,'/global/homes/b/bpb/repos/blink/lbnl_tools/ToM_Diversity/')
import blink_diversity

sys.path.insert(0,'/global/homes/b/bpb/repos/metatlas')
from metatlas.untargeted import tools as mzm


In [2]:
# from importlib import reload  # Python 3.4+
# reload(blink_diversity)

In [3]:
import pandas as pd
md = blink_diversity.metabolite_diversity()
# comparisons = pd.read_pickle(md.params.berklab_mcs_filename)
# comparisons.head()

In [4]:
%%time
md = blink_diversity.metabolite_diversity()
md.polarity = 'Positive'
md.get_treecompound_metadata()
md.make_linkage_from_mcs_matrix()
md.make_skbio_tree()
md.get_berklab_ref()

/global/common/software/m2650/python3-matchms/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/global/homes/b/bpb/repos/blink/lbnl_tools/ToM_Diversity/blink_diversity.py:35: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  z = linkage(n,method='single')


CPU times: user 19.2 s, sys: 578 ms, total: 19.8 s
Wall time: 20 s


In [6]:
%%time
# exp_file = '/project/projectdirs/metatlas/projects/untargeted_tasks/20210920_JGI-AK-TH_NB_504882_SoilWetDry_final_QE-HF_C18_USDAY63680_positive/20210920_JGI-AK-TH_NB_504882_SoilWetDry_final_QE-HF_C18_USDAY63680_positive_MSMS.mgf'
exp_file = '/global/cfs/cdirs/metatlas/projects/untargeted_tasks/20190823_LZH-AG_BM_CondTannins_Kosakonia_0718H2O_QE119_EPC18_USDAY26531_positive/20190823_LZH-AG_BM_CondTannins_Kosakonia_0718H2O_QE119_EPC18_USDAY26531_positive_MSMS.mgf'
md.query_file = exp_file
md.blink_score_file()

# connect it to ToM


md.pd_diversity_file()
md.pd_diversity

CPU times: user 3.62 s, sys: 540 ms, total: 4.16 s
Wall time: 4.25 s


2356.850389606431

In [5]:
%%time
# exp_file = '/global/cfs/cdirs/metatlas/projects/untargeted_tasks/20180426_KBL_DCD_C18_Sorghum_DvsW_RSZ_positive/20180426_KBL_DCD_C18_Sorghum_DvsW_RSZ_positive_MSMS.mgf'
exp_file = '/project/projectdirs/metatlas/projects/untargeted_tasks/20210920_JGI-AK-TH_NB_504882_SoilWetDry_final_QE-HF_C18_USDAY63680_positive/20210920_JGI-AK-TH_NB_504882_SoilWetDry_final_QE-HF_C18_USDAY63680_positive_MSMS.mgf'
md.query_file = exp_file
md.blink_score_file()
md.pd_diversity_file()
md.pd_diversity
# md.num_unique_inchikey_hits
# md.num_unique_inchikey_hits_in_network

CPU times: user 1min 41s, sys: 18.9 s, total: 2min
Wall time: 2min


1856.638837544888

In [23]:
sql = """SELECT Key, parent_dir, mzmine_pos_status,mzmine_neg_status FROM untargeted_tasks
    WHERE mzmine_pos_status='07 complete' AND
    mzmine_neg_status='07 complete';"""

sql_result = mzm.api.query.execute_sql('lists', sql,max_rows=int(1e9))
if len(sql_result['rows'])>0:
    df = pd.DataFrame(sql_result['rows'])
df = df[['Key','parent_dir']]

In [24]:
out = []
for i,row in df.iterrows():
    output_dir='/global/cfs/cdirs/metatlas/projects/untargeted_tasks/'
    mgf_file = mzm.build_untargeted_filename(output_dir,row['parent_dir'],'positive','msms-mzmine')
    temp = None
    if os.path.isfile(mgf_file):
        md.query_file = mgf_file
        md.blink_score_file()
        md.pd_diversity_file()
        temp = md.pd_diversity
    out.append(temp)

KeyboardInterrupt: 

In [27]:
df.reset_index(inplace=True,drop=True)

In [36]:
df.loc[513,'parent_dir']

'20210512_LZH_SK-MdR_SoilExtr_MethodTest_all_QE119_EPC18_USDAY59441'

In [33]:
out[513]

2357.432901285311

In [35]:
np.histogram(out,bins=100)

(array([10,  4,  9,  4,  6,  5,  6,  1,  2,  5,  2,  4,  6,  3,  2,  0,  6,
         8,  6,  5,  8,  3, 10, 12, 10,  7,  8,  2,  9,  9, 10,  9, 13, 13,
        10,  9,  3,  5, 12, 10,  9,  9, 12,  5, 12, 16, 10,  8, 11, 12, 13,
        10, 11,  6,  6, 12,  7,  9, 12,  4,  7,  8,  6,  5,  4, 11,  5, 10,
         9,  5,  6,  8,  4,  3,  2,  4,  2,  6,  1,  0,  1,  5,  2,  1,  2,
         0,  1,  0,  0,  0,  0,  0,  2,  1,  1,  2,  2,  3,  0, 51]),
 array([   0.        ,   23.57432901,   47.14865803,   70.72298704,
          94.29731605,  117.87164506,  141.44597408,  165.02030309,
         188.5946321 ,  212.16896112,  235.74329013,  259.31761914,
         282.89194815,  306.46627717,  330.04060618,  353.61493519,
         377.18926421,  400.76359322,  424.33792223,  447.91225124,
         471.48658026,  495.06090927,  518.63523828,  542.2095673 ,
         565.78389631,  589.35822532,  612.93255433,  636.50688335,
         660.08121236,  683.65554137,  707.22987039,  730.8041994 ,
      

In [30]:
import numpy as np
out = np.asarray(out)
idx = np.argsort(out)[::-1]
df.loc[idx[:10]]

,Key,parent_dir
513,8343,20210512_LZH_SK-MdR_SoilExtr_MethodTest_all_QE...
196,7769,20200218_AG_YH_Niyogi_DiurnalExu_EcoFABFeb_QE1...
370,8142,20181205_JJ_JS_Wildermuth_Medicago4_RtExudates...
375,8147,20200713_AG_YH_Niyogi_DiurnalExu_Set1_QE119_EP...
267,7877,20200818_AG_RM_ExometabProj_PlantExudateExp_G_...
407,8181,20180703_SK_JS_Wildermuth_Diurnal2_171231_QE11...
414,8188,20181107_JJ_KZ_Switchgrass_Greenhouse_Rhizo1_Q...
225,7814,20200819_AG_RM_ExometabProj_PlantExudateExp_W1...
422,8198,20180213_SK_ZZ_ECOFAB_Diurnal_151015_QE119_EPC18
278,7894,20180824_MV_PA_MCAFEs_IntEcoFABExpt_20180817_Q...


In [8]:
# import pandas as pd
# import numpy as np
# import skbio

# diversity = skbio.diversity.alpha.faith_pd(md.hits['present'], md.node_ids, md.ToM)

In [ ]:
# @ray.remote
# def compute_div(exp_file):
#     exp_file = '/global/cfs/cdirs/metatlas/projects/untargeted_tasks/20180426_KBL_DCD_C18_Sorghum_DvsW_RSZ_positive/20180426_KBL_DCD_C18_Sorghum_DvsW_RSZ_positive_MSMS.mgf'
#     md.query_file = exp_file
#     md.blink_score_file()
#     md.pd_diversity_file()
#     md.pd_diversity
#     return jaccard

# futures = [compute_div.remote(f) for f in files]
# mcs_scores = ray.get(futures)